In [66]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [67]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [68]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import warnings
warnings.filterwarnings("ignore")

In [69]:
import sys
sys.path.append("../../") 

from utils.paths import make_dir_line

modality = 'c'
project = 'Introduction to Relational Databases in SQL'
data = make_dir_line(modality, project)

raw = data('raw')

In [70]:
import sqlite3

conn = sqlite3.connect(":memory:")  ## aca se indica el nombre de la db.
cur = conn.cursor()

# 6.2.0 Enforce data consistency with attribute constraints

info: https://www.postgresql.org/docs/10/datatype-datetime.html#DATATYPE-DATETIME-INPUT

In [71]:
conn.executescript(
    """
    
DROP TABLE IF EXISTS transactions;

CREATE TABLE transactions (
  transaction_date    DATE,
  amount              INTEGER,
  fee                 TEXT
);

"""
)
conn.commit()

In [72]:
df = pd.read_csv(raw / 'transactions.csv', sep = ',', decimal = '.', header = 0, encoding = 'utf-8')
transactions = list(zip(df.transaction_date, df.amount, df.fee))
cur.executemany("INSERT INTO transactions VALUES  (?,?,?)", transactions)

## 6.2.2 Types of database constraints

Which of the following is **not** used to enforce a database constraint?

R:/ SQL aggregate functions. SQL aggregate functions are not used to enforce constraints, but to do calculations on data.

## 6.2.3 Conforming with data types

In [73]:
# -- Let's add a record to the table
cur.execute("""
            
            INSERT INTO transactions (transaction_date, amount, fee) 
            VALUES ('24/09/2018', 5454, '30');
                        
            """).fetchall()

[]

In [74]:
# -- Doublecheck the contents
cur.execute("""
            
            SELECT *
            FROM transactions;
                        
            """).fetchall()

[('8/01/1999', 500, '20'),
 ('20/02/2001', 403, '15'),
 ('20/03/2001', 3430, '35'),
 ('24/09/2018', 5454, '30')]

## 6.2.4 Type CASTs

In [75]:
# -- Calculate the net amount as amount + fee
cur.execute("""
            
            SELECT transaction_date, amount + CAST(fee AS integer) AS net_amount 
            FROM transactions;
                        
            """).fetchall()

[('8/01/1999', 520),
 ('20/02/2001', 418),
 ('20/03/2001', 3465),
 ('24/09/2018', 5484)]

## 6.2.6 Change types with ALTER COLUMN

In [76]:
conn = sqlite3.connect(":memory:") 
cur = conn.cursor()

In [77]:
conn.executescript(
    """
    
    DROP TABLE IF EXISTS university_professors;

    CREATE TABLE university_professors (
        firstname             VARCHAR,
        lastname              VARCHAR,
        university            VARCHAR,
        university_shortname  VARCHAR,
        university_city       VARCHAR,
        function              VARCHAR,
        organization          VARCHAR,
        organization_sector   VARCHAR
    );


    DROP TABLE IF EXISTS professors;

    CREATE TABLE professors (
        firstname            text,
        lastname             text,
        university_shortname text
    );


    DROP TABLE IF EXISTS universities;
    
    CREATE TABLE universities (
        university_shortname text,
        university           text,
        university_city      text
    );


    DROP TABLE IF EXISTS organizations;

    CREATE TABLE organizations (
        organization        text,
        organization_sector text
    );


    DROP TABLE IF EXISTS affiliations;

    CREATE TABLE affiliations (
        firstname             text,
        lastname              text,
        function              text,
        organization          text
    );


    """
)
conn.commit()

In [78]:
df = pd.read_csv(raw / 'university_professors.csv', sep = ',', decimal = '.', header = 0, encoding = 'utf-8')
university_professors = list(zip(df.firstname, df.lastname, df.university, df.university_shortname, df.university_city, df.function, df.organization, df.organization_sector))
cur.executemany("INSERT INTO university_professors VALUES  (?,?,?,?,?,?,?,?)", university_professors)

In [79]:
cur.executescript("""
    
    INSERT INTO professors 
    SELECT DISTINCT firstname, lastname, university_shortname 
    FROM university_professors;

    INSERT INTO affiliations 
    SELECT DISTINCT firstname, lastname, function, organization 
    FROM university_professors;            
                
                """)

In [80]:
# -- Select the university_shortname column
cur.execute("""
            
            SELECT DISTINCT(university_shortname) 
            FROM professors;
                        
            """).fetchall()

[('EPF',),
 ('ETH',),
 ('UBE',),
 ('ULA',),
 ('USG',),
 ('UNE',),
 ('UFR',),
 ('USI',),
 ('UBA',),
 ('UGE',),
 ('UZH',)]

In [81]:
# # -- Specify the correct fixed-length character type
# cur.execute("""
            
#             ALTER TABLE professors
#             ALTER COLUMN university_shortname
#             TYPE char(3);
                        
#             """).fetchall()

In [82]:
# # -- Specify the correct fixed-length character type
# cur.execute("""
            
#             ALTER TABLE professors
#             ALTER COLUMN firstname
#             TYPE varchar(64);
                        
#             """).fetchall()

## 6.2.7 Convert types USING a function

If you don't want to reserve too much space for a certain varchar column, you can truncate the values before converting its type.

In [83]:
# # -- Convert the values in firstname to a max. of 16 characters
# cur.execute("""
            
#             ALTER TABLE professors 
#             ALTER COLUMN firstname 
#             TYPE varchar(16)
#             USING SUBSTRING(firstname FROM 1 FOR 16)
                        
#             """).fetchall()

## 6.2.9 Disallow NULL values with SET NOT NULL

In [84]:
# # -- Disallow NULL values in firstname
# cur.execute("""
            
#             ALTER TABLE professors 
#             ALTER COLUMN firstname SET NOT NULL;
                        
#             """).fetchall()

In [85]:
# # -- Disallow NULL values in lastname
# cur.execute("""
                        
#             ALTER TABLE professors 
#             ALTER COLUMN lastname SET NOT NULL;
                        
#             """).fetchall()

## 6.2.10 What happens if you try to enter NULLs?

In [86]:
# cur.execute("""
            
#             INSERT INTO professors (firstname, lastname, university_shortname)
#             VALUES (NULL, 'Miller', 'ETH');
                        
#             """).fetchall()

Question: Why does this throw an error?

R:/ Because a database constraint is violated. This statement violates one of the not-null constraints you've just specified.

## 6.2.11 Make your columns UNIQUE with ADD CONSTRAINT

In [87]:
# -- Make universities.university_shortname unique
# cur.execute("""
            
#             ALTER TABLE universities
#             ADD CONSTRAINT university_shortname_unq UNIQUE(university_shortname);
                        
#             """).fetchall()

In [88]:
# # -- Make organizations.organization unique
# cur.execute("""
            
#             ALTER TABLE organizations
#             ADD CONSTRAINT organization_unq UNIQUE(organization);
                        
#             """).fetchall()

In [89]:
print('Ok_')

Ok_
